<img src="images/splash3.png">
Grabs price and volume for the top 100 cryptos for the past 3 months.

OOP update#2

## Setup

In [1]:
import sqlite3 as lite
import sys
import coinmarketcap
import requests
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("display.max_rows", 200)

## Fetch data
Uses cryptocompare API to get historical data for target coin the last 3 months, coinmarketcap api to get top-100 coin names. Timestamps are in unix (GMT).

In [2]:
def fetch_top100():
    url = 'https://api.coinmarketcap.com/v2/ticker/'
    page = requests.get(url)
    data = page.json()['data']
    coinlist = pd.DataFrame(data)
    return coinlist

In [3]:
def fetch_coindata(symbol, comparison_symbol, limit, aggregate):    
    url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)       
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['mid'] = df[["high", "low"]].mean(axis=1)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d).strftime('%d-%m-%Y %H:%M:%S') for d in df.time]
    #df = df.set_index('timestamp')        
    return df

## Store values to database

In [4]:
def write_to_db():
    #Connect to DB
    con = None
    con = lite.connect('/hdd/db/crypto.db')

    #Dump to SQL
    with con:
        market_data.to_sql(name=top_coins[symbol], con=con, if_exists='replace', index=True)

    """"
    #Read SQL    
    with con:        
        cur = con.cursor()    
        cur.execute("SELECT * FROM btc")

        rows = cur.fetchall()

        for row in rows:
            print(row)  
    """

In [5]:
top_coins = fetch_top100().loc['symbol']

for symbol in range(1, 100):
    try:
        market_data = fetch_coindata(top_coins[symbol],'BTC','2000','12')
        market_data = market_data.drop(market_data.index[len(market_data)-1]) # Only keep the latest timestamped column
        write_to_db()
    except:
        print("Error noticed, skipping coin ", top_coins[symbol])
        
print("Success!")

Error noticed, skipping coin  MIOTA
Error noticed, skipping coin  CENNZ
Success!


In [6]:
market_data

,close,high,low,open,time,volumefrom,volumeto,mid,timestamp
0,0.000034,0.000036,0.000034,0.000034,1521417600,3.131684e+07,1081.88,0.000035,19-03-2018 01:00:00
1,0.000035,0.000036,0.000031,0.000034,1521460800,7.559852e+07,2518.36,0.000034,19-03-2018 13:00:00
2,0.000039,0.000041,0.000032,0.000034,1521504000,1.352212e+08,4963.87,0.000036,20-03-2018 01:00:00
3,0.000034,0.000039,0.000033,0.000039,1521547200,7.237028e+07,2554.28,0.000036,20-03-2018 13:00:00
4,0.000034,0.000035,0.000033,0.000034,1521590400,5.661058e+07,1921.20,0.000034,21-03-2018 01:00:00
5,0.000032,0.000034,0.000032,0.000034,1521633600,3.483557e+07,1148.52,0.000033,21-03-2018 13:00:00
6,0.000033,0.000034,0.000032,0.000032,1521676800,3.055480e+07,1011.33,0.000033,22-03-2018 01:00:00
7,0.000034,0.000035,0.000032,0.000033,1521720000,2.033199e+07,685.81,0.000034,22-03-2018 13:00:00
8,0.000032,0.000034,0.000032,0.000034,1521763200,1.442959e+07,475.13,0.000033,23-03-2018 01:00:00
9,0.000031,0.000033,0.000031,0.000032,1521806400,1.987574e+07,631.06,0.000032,23-03-2018 13:00:00
